In [1]:
import argparse
import random
import pandas as pd
import warnings
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
from model.model_lora import *
from trainer.trainer_utils import setup_seed
from torch.utils.data import Dataset
warnings.filterwarnings('ignore')

d:\Anaconda\envs\py312pt291cu128\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv('dataset/bbc-news-data.csv',on_bad_lines='warn',encoding='utf-8',sep='\t')

In [3]:
data=[]
for index,row in df.iterrows():
    data.append(row[df.columns[0]])
print(data)

['business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'business', 'bu

In [3]:
import random
index = [i for i in range(len(df))]
random.shuffle(index)
df = df.iloc[index]

In [6]:
import re
def split_text_into_chunks(text, target_words=700, overlap_words=50):
    """
    将文本切分成指定单词数的chunks，尽量在句子边界处切分
    
    Args:
        text: 要切分的文本
        target_words: 目标单词数（约）
        overlap_words: chunk之间的重叠单词数
    
    Returns:
        chunks列表
    """
    if not text or not text.strip():
        return []
    
    # 按空格分割单词
    words = text.split()
    if len(words) <= target_words:
        return [text]
    
    chunks = []
    start_idx = 0
    
    while start_idx < len(words):
        # 计算当前chunk的结束位置
        end_idx = min(start_idx + target_words, len(words))
        
        # 如果还没到文本末尾，尝试在句子边界处切分
        if end_idx < len(words):
            # 向后查找句子结束符（. ! ? 后跟空格或换行）
            # 在目标位置前后100个单词范围内查找
            search_start = max(start_idx + target_words - 100, start_idx)
            search_end = min(end_idx + 100, len(words))
            
            best_split = end_idx
            # 从目标位置向前查找句子边界
            for i in range(end_idx, search_start, -1):
                if i < len(words):
                    # 检查前一个单词是否以句子结束符结尾
                    prev_word = words[i-1] if i > 0 else ""
                    if prev_word and re.search(r'[.!?]$', prev_word):
                        best_split = i
                        break
            
            end_idx = best_split
        
        # 提取chunk
        chunk_words = words[start_idx:end_idx]
        chunk_text = " ".join(chunk_words)
        
        if chunk_text.strip():
            chunks.append(chunk_text.strip())
        
        # 移动到下一个chunk的起始位置（考虑重叠）
        if end_idx >= len(words):
            break
        start_idx = max(end_idx - overlap_words, start_idx + 1)
    
    return chunks

def generate_prompt(title,content):
    chunks=split_text_into_chunks(content,200,100)
    print(len(chunks))
    response=[]
    for i in chunks:
        response.append(f"title:{title}\nbody:{i}\naccording to the content, please classify the article into one of the following categories: business, entertainment, politics, sport, technology.\nanswer: ")
    return response
class dataset(Dataset):
    def __init__(self, data):
        self.data=data
        self.prompt = []
        self.label=[]
        for i in range(len(data)):
            response=generate_prompt(data.at[i,'title'],data.at[i,'content'])
            self.prompt.extend(response)
            self.label.extend([data.at[i,'category']]*len(response))
    def __len__(self):
        return len(self.prompt)
df1=df[:1113]
df2=df[1113:]
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
train_dataset=dataset(df1)


5
5
2
9
3
4
3
4
6
1
2
2
1
2
2
2
2
4
4
11
3
3
1
1
2
4
2
2
4
3
5
4
3
5
7
4
3
4
2
4
2
2
1
2
3
4
8
3
10
4
3
2
2
2
3
2
8
3
3
2
2
6
5
4
2
2
3
5
2
3
1
3
1
3
7
1
5
2
7
1
2
5
9
5
4
7
4
10
2
2
1
4
3
5
2
6
4
5
3
2
4
7
3
6
9
1
2
6
2
9
3
3
6
9
5
3
3
1
6
3
2
3
5
4
2
2
2
6
6
1
2
2
4
2
4
1
7
4
2
2
2
2
2
4
3
5
2
2
3
3
9
2
2
6
1
3
2
1
5
2
5
3
2
6
3
1
2
2
4
5
3
2
5
5
5
5
2
4
2
3
2
1
2
1
4
2
7
3
3
3
4
1
4
1
2
5
8
3
3
5
3
2
1
4
1
3
1
6
2
4
5
1
2
2
1
4
3
4
5
3
2
6
4
2
2
10
3
3
7
3
2
3
4
1
2
2
2
4
3
2
1
3
1
5
1
3
3
5
1
3
2
2
3
4
4
1
5
4
2
6
16
8
6
2
5
4
4
20
3
2
3
2
2
6
3
1
1
4
5
2
6
4
6
5
2
3
12
2
3
6
2
1
2
1
7
1
3
4
2
1
2
5
6
2
2
2
1
2
5
1
4
5
4
9
4
2
2
3
2
2
5
10
2
3
4
2
3
7
4
5
2
4
2
4
3
1
6
2
9
8
2
3
4
10
5
1
3
1
3
5
4
2
3
3
1
6
3
3
2
2
2
4
6
4
2
2
3
3
4
3
3
1
3
4
3
3
3
9
4
9
3
2
2
3
4
2
3
3
4
1
2
4
8
2
2
2
2
5
2
4
2
1
1
1
6
3
5
2
1
3
3
3
2
3
2
5
26
3
3
6
11
3
4
8
3
4
3
3
1
1
2
3
4
1
2
4
3
3
2
1
3
3
3
3
2
6
3
5
2
7
3
2
9
3
4
3
7
3
6
5
2
7
3
15
3
3
6
5
3
1
4
1
5
3
4
7
5
4
2
2
2
6
2
3
7
4
1
2
2
5
3
3
6
3


In [11]:
def generate_prompt(title,content):
    # chunks=split_text_into_chunks(content,200,100)
    # print(len(chunks))
    # response=[]
    # for i in chunks:
    #     response.append(f"title:{title}\nbody:{i}\naccording to the content, please classify the article into one of the following categories: business, entertainment, politics, sport, technology.\nanswer: ")
    return f"title:{title}\nbody:{content}\naccording to the content, please classify the article into one of the following categories: business, entertainment, politics, sport, technology.\nanswer: "
class dataset(Dataset):
    def __init__(self, data):
        self.data=data
        self.prompt = []
        self.label=[]
        for i in range(len(data)):
            response=generate_prompt(data.at[i,'title'],data.at[i,'content'])
            self.prompt.append(response)
            self.label.append(data.at[i,'category'])
    def __len__(self):
        return len(self.prompt)
test_dataset=dataset(df2)

In [12]:
print(len(test_dataset))

1112


In [13]:
# 将训练集转换为jsonl格式（参考lora_identity.jsonl的结构）
import json

def save_dataset_to_jsonl(dataset, output_file):
    """
    将dataset转换为jsonl格式并保存
    
    Args:
        dataset: dataset对象，包含prompt和label属性
        output_file: 输出文件路径
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        for i in range(len(dataset)):
            # 构建conversations格式
            conversation = {'text':dataset.prompt[i],'label':dataset.label[i]}
            # 写入jsonl文件（每行一个JSON对象）
            f.write(json.dumps(conversation, ensure_ascii=False) + '\n')
    print(f"已保存 {len(dataset)} 条数据到 {output_file}")
# save_dataset_to_jsonl(full_dataset, 'dataset/bbc_news_data_no_instruction.jsonl')
# 保存训练集
save_dataset_to_jsonl(test_dataset, 'dataset/bbc_news_test_en.jsonl')

# # 保存测试集（可选）
# save_dataset_to_jsonl(test_dataset, 'dataset/bbc_news_test_no_instruction.jsonl')

# # 验证：读取前几行查看格式
# print("\n验证格式（前3条）：")
# with open('dataset/bbc_news_train.jsonl', 'r', encoding='utf-8') as f:
#     for i, line in enumerate(f):
#         if i >= 3:
#             break
#         data = json.loads(line)
#         print(f"\n第{i+1}条:")
#         print(f"  User: {data['conversations'][0]['content'][:100]}...")
#         print(f"  Assistant: {data['conversations'][1]['content']}")


已保存 1112 条数据到 dataset/bbc_news_test_en.jsonl


In [1]:
# 基础检查：不需要tokenizer，只检查JSON格式和结构
import json

def check_training_data_basic(jsonl_path):
    """
    基础检查：不需要tokenizer，只检查JSON格式和结构
    """
    print(f"正在检查数据文件: {jsonl_path}")
    print("=" * 80)
    
    stats = {
        'total_samples': 0,
        'json_errors': [],
        'empty_conversations': [],
        'invalid_structure': [],
        'empty_content': []
    }
    
    print("\n[基础检查] JSON格式和基本结构...")
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            stats['total_samples'] += 1
            try:
                data = json.loads(line.strip())
                
                if 'conversations' not in data:
                    stats['invalid_structure'].append({
                        'line': line_num,
                        'error': 'Missing "conversations" field'
                    })
                    continue
                
                if not data['conversations'] or len(data['conversations']) == 0:
                    stats['empty_conversations'].append({'line': line_num})
                    continue
                
                for i, conv in enumerate(data['conversations']):
                    if 'role' not in conv or 'content' not in conv:
                        stats['invalid_structure'].append({
                            'line': line_num,
                            'error': f'Conversation {i} missing "role" or "content"'
                        })
                        break
                    if not conv['content'] or len(conv['content'].strip()) == 0:
                        stats['empty_content'].append({
                            'line': line_num,
                            'conversation_index': i,
                            'role': conv.get('role', 'unknown')
                        })
                
            except json.JSONDecodeError as e:
                stats['json_errors'].append({
                    'line': line_num,
                    'error': str(e)
                })
    
    print(f"  总样本数: {stats['total_samples']}")
    print(f"  JSON格式错误: {len(stats['json_errors'])}")
    print(f"  空conversations: {len(stats['empty_conversations'])}")
    print(f"  结构错误: {len(stats['invalid_structure'])}")
    print(f"  空内容: {len(stats['empty_content'])}")
    
    if stats['json_errors'] or stats['invalid_structure'] or stats['empty_conversations']:
        print("\n⚠️  发现基础问题，请先修复！")
    else:
        print("\n✅ 基础检查通过！")
    
    return stats

# 先执行基础检查（不需要tokenizer）
basic_stats = check_training_data_basic('dataset/bbc_news_train.jsonl')


正在检查数据文件: dataset/bbc_news_train.jsonl

[基础检查] JSON格式和基本结构...
  总样本数: 1113
  JSON格式错误: 0
  空conversations: 0
  结构错误: 0
  空内容: 0

✅ 基础检查通过！


In [8]:
# 完整检查：需要tokenizer，检查loss_mask（这是导致nan的关键）
# 注意：需要先运行Cell 10初始化model和tokenizer
from dataset.lm_dataset import SFTDataset

def check_training_data_full(jsonl_path, tokenizer, max_length=10240):
    """
    完整检查：需要tokenizer，检查loss_mask（这是导致nan的关键）
    
    检查项：
    1. JSON格式是否正确
    2. conversations结构是否完整
    3. loss_mask是否全为0（会导致除零错误）
    4. tokenized后的长度是否异常
    5. 是否有空内容
    """
    print(f"正在完整检查数据文件: {jsonl_path}")
    print("=" * 80)
    
    # 统计信息
    stats = {
        'total_samples': 0,
        'json_errors': [],
        'empty_conversations': [],
        'invalid_structure': [],
        'zero_loss_mask': [],
        'too_long': [],
        'too_short': [],
        'empty_content': [],
        'special_chars_issues': []
    }
    
    # 1. 检查JSON格式和基本结构
    print("\n[1] 检查JSON格式和基本结构...")
    valid_samples = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            stats['total_samples'] += 1
            try:
                data = json.loads(line.strip())
                
                if 'conversations' not in data:
                    stats['invalid_structure'].append({
                        'line': line_num,
                        'error': 'Missing "conversations" field'
                    })
                    continue
                
                if not data['conversations'] or len(data['conversations']) == 0:
                    stats['empty_conversations'].append({
                        'line': line_num,
                        'data': data
                    })
                    continue
                
                for i, conv in enumerate(data['conversations']):
                    if 'role' not in conv or 'content' not in conv:
                        stats['invalid_structure'].append({
                            'line': line_num,
                            'error': f'Conversation {i} missing "role" or "content"'
                        })
                        break
                    if not conv['content'] or len(conv['content'].strip()) == 0:
                        stats['empty_content'].append({
                            'line': line_num,
                            'conversation_index': i,
                            'role': conv.get('role', 'unknown')
                        })
                
                valid_samples.append((line_num, data))
                
            except json.JSONDecodeError as e:
                stats['json_errors'].append({
                    'line': line_num,
                    'error': str(e),
                    'content': line[:100]
                })
    
    print(f"  总样本数: {stats['total_samples']}")
    print(f"  JSON格式错误: {len(stats['json_errors'])}")
    print(f"  空conversations: {len(stats['empty_conversations'])}")
    print(f"  结构错误: {len(stats['invalid_structure'])}")
    print(f"  空内容: {len(stats['empty_content'])}")
    
    # 2. 使用SFTDataset检查loss_mask（这是导致nan的关键）
    print("\n[2] 检查loss_mask（这是导致nan的关键）...")
    try:
        dataset = SFTDataset(jsonl_path, tokenizer, max_length=max_length)
        
        for idx in range(len(dataset)):
            try:
                X, Y, loss_mask = dataset[idx]
                
                # 检查loss_mask是否全为0
                loss_mask_sum = loss_mask.sum().item()
                if loss_mask_sum == 0:
                    stats['zero_loss_mask'].append({
                        'index': idx,
                        'line': valid_samples[idx][0] if idx < len(valid_samples) else 'unknown',
                        'X_shape': X.shape,
                        'Y_shape': Y.shape
                    })
                
                # 检查长度
                seq_len = X.shape[0]
                if seq_len < 10:
                    stats['too_short'].append({
                        'index': idx,
                        'line': valid_samples[idx][0] if idx < len(valid_samples) else 'unknown',
                        'length': seq_len
                    })
                elif seq_len >= max_length - 1:
                    stats['too_long'].append({
                        'index': idx,
                        'line': valid_samples[idx][0] if idx < len(valid_samples) else 'unknown',
                        'length': seq_len
                    })
                    
            except Exception as e:
                stats['special_chars_issues'].append({
                    'index': idx,
                    'line': valid_samples[idx][0] if idx < len(valid_samples) else 'unknown',
                    'error': str(e)
                })
                
    except Exception as e:
        print(f"  错误: 无法创建dataset - {e}")
    
    # 3. 打印详细报告
    print("\n" + "=" * 80)
    print("详细异常报告:")
    print("=" * 80)
    
    if stats['json_errors']:
        print(f"\n❌ JSON格式错误 ({len(stats['json_errors'])} 个):")
        for err in stats['json_errors'][:5]:
            print(f"  行 {err['line']}: {err['error']}")
    
    if stats['zero_loss_mask']:
        print(f"\n⚠️  关键问题: loss_mask全为0 ({len(stats['zero_loss_mask'])} 个)")
        print("   这会导致训练时除零错误，产生nan！")
        for err in stats['zero_loss_mask'][:10]:
            print(f"  样本索引 {err['index']} (行 {err['line']})")
            if err['index'] < len(valid_samples):
                line_num, data = valid_samples[err['index']]
                print(f"    conversations数量: {len(data['conversations'])}")
                for i, conv in enumerate(data['conversations']):
                    content_preview = conv['content'][:50] + "..." if len(conv['content']) > 50 else conv['content']
                    print(f"      [{i}] {conv['role']}: {content_preview}")
    
    if stats['empty_conversations']:
        print(f"\n❌ 空conversations ({len(stats['empty_conversations'])} 个):")
        for err in stats['empty_conversations'][:5]:
            print(f"  行 {err['line']}")
    
    if stats['invalid_structure']:
        print(f"\n❌ 结构错误 ({len(stats['invalid_structure'])} 个):")
        for err in stats['invalid_structure'][:5]:
            print(f"  行 {err['line']}: {err['error']}")
    
    if stats['empty_content']:
        print(f"\n⚠️  空内容 ({len(stats['empty_content'])} 个):")
        for err in stats['empty_content'][:5]:
            print(f"  行 {err['line']}, conversation {err['conversation_index']}, role: {err['role']}")
    
    if stats['too_short']:
        print(f"\n⚠️  序列过短 ({len(stats['too_short'])} 个):")
        for err in stats['too_short'][:5]:
            print(f"  样本索引 {err['index']} (行 {err['line']}): 长度 {err['length']}")
    
    if stats['too_long']:
        print(f"\n⚠️  序列过长 ({len(stats['too_long'])} 个):")
        for err in stats['too_long'][:5]:
            print(f"  样本索引 {err['index']} (行 {err['line']}): 长度 {err['length']}")
    
    if stats['special_chars_issues']:
        print(f"\n⚠️  特殊字符问题 ({len(stats['special_chars_issues'])} 个):")
        for err in stats['special_chars_issues'][:5]:
            print(f"  样本索引 {err['index']} (行 {err['line']}): {err['error']}")
    
    # 4. 总结
    print("\n" + "=" * 80)
    print("检查总结:")
    print("=" * 80)
    total_issues = (len(stats['json_errors']) + 
                   len(stats['empty_conversations']) + 
                   len(stats['invalid_structure']) + 
                   len(stats['zero_loss_mask']) + 
                   len(stats['empty_content']))
    
    if total_issues == 0:
        print("✅ 未发现明显异常！")
    else:
        print(f"⚠️  发现 {total_issues} 个潜在问题")
        if stats['zero_loss_mask']:
            print(f"\n🔴 严重: 有 {len(stats['zero_loss_mask'])} 个样本的loss_mask全为0")
            print("   这些样本会导致训练时出现nan，建议修复或过滤！")
    
    return stats

full_stats = check_training_data_full(
    jsonl_path='dataset/bbc_news_train.jsonl',
    tokenizer=tokenizer,
    max_length=512
)


正在完整检查数据文件: dataset/bbc_news_train.jsonl

[1] 检查JSON格式和基本结构...
  总样本数: 1113
  JSON格式错误: 0
  空conversations: 0
  结构错误: 0
  空内容: 0

[2] 检查loss_mask（这是导致nan的关键）...

详细异常报告:

⚠️  关键问题: loss_mask全为0 (901 个)
   这会导致训练时除零错误，产生nan！
  样本索引 0 (行 1)
    conversations数量: 2
      [0] user: 标题：Fockers fuel festive film chart
内容： Comedy Meet...
      [1] assistant: entertainment
  样本索引 2 (行 3)
    conversations数量: 2
      [0] user: 标题：'Hitler' row over Welsh arts cash
内容： An artist...
      [1] assistant: politics
  样本索引 3 (行 4)
    conversations数量: 2
      [0] user: 标题：Davenport hits out at Wimbledon
内容： World numbe...
      [1] assistant: sport
  样本索引 5 (行 6)
    conversations数量: 2
      [0] user: 标题：Straw to attend Auschwitz service
内容： Foreign S...
      [1] assistant: politics
  样本索引 6 (行 7)
    conversations数量: 2
      [0] user: 标题：US state acts to stop 'spammers'
内容： US state T...
      [1] assistant: tech
  样本索引 7 (行 8)
    conversations数量: 2
      [0] user: 标题：A-listers flock to Gervais sitc

In [23]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args = argparse.Namespace(
        load_from='model',
        save_dir='out',
        weight='full_sft',
        lora_weight='None',
        hidden_size=512,
        num_hidden_layers=8,
        use_moe=0,
        inference_rope_scaling=False,
        max_new_tokens=8192,
        temperature=0.85,
        top_p=0.85,
        historys=0,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
def init_model(args):
    tokenizer = AutoTokenizer.from_pretrained(args.load_from)
    if 'model' in args.load_from:
        model = MiniMindForCausalLM(MiniMindConfig(
            hidden_size=args.hidden_size,
            num_hidden_layers=args.num_hidden_layers,
            use_moe=bool(args.use_moe),
            inference_rope_scaling=args.inference_rope_scaling
        ))
        moe_suffix = '_moe' if args.use_moe else ''
        ckp = f'./{args.save_dir}/{args.weight}_{args.hidden_size}{moe_suffix}.pth'
        model.load_state_dict(torch.load(ckp, map_location=args.device), strict=True)
        if args.lora_weight != 'None':
            apply_lora(model)
            load_lora(model, f'./{args.save_dir}/lora/{args.lora_weight}_{args.hidden_size}.pth')
    else:
        model = AutoModelForCausalLM.from_pretrained(args.load_from, trust_remote_code=True)
    print(f'MiniMind模型参数: {sum(p.numel() for p in model.parameters()) / 1e6:.2f} M(illion)')
    return model.eval().to(device), tokenizer
model, tokenizer = init_model(args)

MiniMind模型参数: 25.83 M(illion)


In [4]:
sentence='''I love you so much'''
encoded=tokenizer.encode(sentence,add_special_tokens=False)
print(encoded)
for i in encoded:
    print(tokenizer.decode([i]),end='|')
# decoded=tokenizer.decode(encoded)
# print(decoded)


[43, 2344, 363, 1059, 3067]
I| love| you| so| much|

In [ ]:
vocab=tokenizer.get_vocab()
print(sorted(vocab,key=lambda x:vocab[x]))


['<|endoftext|>', '<|im_start|>', '<|im_end|>', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', '÷', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ', 'ÿ', 'Ā', 'ā', '

In [24]:
from peft import PromptEmbedding, PromptTuningConfig,get_peft_model
import torch.nn as nn
model, tokenizer = init_model(args)
peft_config = PromptTuningConfig(
    task_type="CAUSAL_LM",
    num_virtual_tokens=20,
    prompt_tuning_init="TEXT",
    num_transformer_submodules=1,
    token_dim=512,
    prompt_tuning_init_text="Classify the passage:",
    tokenizer_name_or_path='./model'
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # 应显示 ~15K 可训练参数



MiniMind模型参数: 25.83 M(illion)
trainable params: 10,240 || all params: 25,840,128 || trainable%: 0.0396


In [25]:
input_prompt = ["Classify the sentiment of the following review as positive or negative.\nReview: I love this film!\nSentiment:"]
inputs = tokenizer(input_prompt, return_tensors="pt").to(device)
print(model.model(inputs['input_ids']))
outputs = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=inputs['attention_mask'],
    max_new_tokens=5,
    pad_token_id=tokenizer.eos_token_id
)
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(prediction)

(tensor([[[-0.7488, -0.0540,  0.1875,  ..., -4.4258,  0.8613, -0.9082],
         [-0.4753,  0.7772,  0.0536,  ..., -0.0755,  1.1915, -0.8722],
         [ 0.3381,  2.9204,  1.4160,  ..., -1.0023,  1.0548, -0.8825],
         ...,
         [ 0.8371,  1.6740, -1.3006,  ..., -0.7171, -3.4437, -0.4352],
         [-0.3711,  1.4281,  0.0099,  ..., -0.6652, -2.6028, -0.7404],
         [-0.6661,  0.9061, -0.4236,  ..., -2.7374, -0.9953, -0.8959]]],
       device='cuda:0'), [None, None, None, None, None, None, None, None], 0)


AttributeError: 'NoneType' object has no attribute 'shape'

In [45]:
# 1. 获取要修改的token的id
token_str = "<|im_start|>"  # 或者任何你想修改的token
token_id = tokenizer.convert_tokens_to_ids(token_str)
print(f"Token '{token_str}' 的 ID: {token_id}")

# 2. 访问embedding层
# 注意：MiniMind中embed_tokens和lm_head共享权重
embedding_layer = model.model.embed_tokens  # 或者 model.lm_head (它们共享权重)

# 3. 查看当前embedding
current_embedding = embedding_layer.weight[token_id]
print(f"当前embedding形状: {current_embedding.shape}")
print(f"当前embedding前5个值: {current_embedding[:5]}")

# # 4. 修改embedding（需要先设置为训练模式或使用requires_grad_）
model.model.embed_tokens.weight.requires_grad_(True)

# # 方法A: 直接赋值新的embedding向量
# new_embedding = torch.randn(model.config.hidden_size)  # 随机初始化
# # 或者从其他token复制
# # new_embedding = embedding_layer.weight[other_token_id].clone()

# embedding_layer.weight.data[token_id] = new_embedding

# # 方法B: 在现有embedding基础上调整
# # embedding_layer.weight.data[token_id] += 0.1 * torch.randn(model.config.hidden_size)

# # 方法C: 设置为特定值
embedding_layer.weight.data[token_id] = torch.zeros(model.config.hidden_size)

# # 5. 验证修改
modified_embedding = embedding_layer.weight[token_id]
print(f"修改后embedding前5个值: {modified_embedding[:5]}")

Token '<|im_start|>' 的 ID: 1
当前embedding形状: torch.Size([512])
当前embedding前5个值: tensor([ 0.0036, -0.0128, -0.0255,  0.0096, -0.0007], device='cuda:0',
       grad_fn=<SliceBackward0>)
修改后embedding前5个值: tensor([0., 0., 0., 0., 0.], device='cuda:0', grad_fn=<SliceBackward0>)


In [46]:
print(tokenizer.bos_token)

<|im_start|>


In [16]:
# 给tokenizer的vocab添加新词
# 方法：使用 tokenizer.add_tokens() 添加新词

# 1. 定义要添加的新词
new_tokens = ["<cls>"]

# 2. 检查这些词是否已经存在于vocab中
existing_tokens = []
for token in new_tokens:
    if token in tokenizer.get_vocab():
        existing_tokens.append(token)
        print(f"⚠️  '{token}' 已经存在于vocab中，ID: {tokenizer.convert_tokens_to_ids(token)}")
    else:
        print(f"✅ '{token}' 不在vocab中，可以添加")

# 3. 过滤掉已存在的词
tokens_to_add = [token for token in new_tokens if token not in existing_tokens]

if tokens_to_add:
    # 4. 添加新词到tokenizer
    num_added = tokenizer.add_tokens(tokens_to_add)
    print(f"\n成功添加 {num_added} 个新词到tokenizer")
    
    # 5. 查看新添加的词及其ID
    print("\n新添加的词及其ID:")
    for token in tokens_to_add:
        token_id = tokenizer.convert_tokens_to_ids(token)
        print(f"  '{token}' -> ID: {token_id}")
    
    # 6. 如果模型需要支持这些新词，需要调整模型嵌入层大小
    # 注意：这需要重新初始化新token的embedding权重
    if num_added > 0:
        print(f"\n⚠️  注意：模型嵌入层需要调整大小以支持新词")
        print(f"当前vocab大小: {len(tokenizer)}")
        print(f"模型embedding层大小: {model.get_input_embeddings().weight.shape[0]}")
        # 在 resize 之前保存原有 embedding
        old_embedding = model.get_input_embeddings().weight.data[:6400].clone()

        # 执行 resize
        model.resize_token_embeddings(len(tokenizer))

        # 比较 resize 后的前 6400 个 embedding
        new_embedding = model.get_input_embeddings().weight.data[:6400]

        # 检查是否相同
        print('是否相同：',torch.allclose(old_embedding, new_embedding))  # 应该返回 True
        # 调整模型嵌入层大小
        # model.resize_token_embeddings(len(tokenizer))
        print(f"调整后模型embedding层大小: {model.get_input_embeddings().weight.shape[0]}")
        
        # 7. 可选：初始化新token的embedding（可以随机初始化或从其他token复制）
        # 获取新添加的token IDs
        new_token_ids = [tokenizer.convert_tokens_to_ids(token) for token in tokens_to_add]
        
        # 方法A: 随机初始化（使用正态分布）
        embedding_dim = model.get_input_embeddings().weight.shape[1]
        for token_id in new_token_ids:
            # 从标准正态分布随机初始化
            new_embedding = torch.randn(embedding_dim) * 0.02  # 小方差初始化
            model.get_input_embeddings().weight.data[token_id] = new_embedding
            # 如果lm_head和embed_tokens共享权重，只需要修改一处
            if hasattr(model, 'lm_head') and model.lm_head.weight is model.get_input_embeddings().weight:
                pass  # 已经共享，无需额外操作
        
        print(f"✅ 已初始化 {len(new_token_ids)} 个新token的embedding")
        
        # 8. 验证新词可以正常使用
        print("\n验证新词的使用:")
        test_text = f"这是一个测试: {tokens_to_add[0]} 应该被正确识别"
        encoded = tokenizer.encode(test_text, add_special_tokens=False)
        decoded = tokenizer.decode(encoded)
        print(f"  原文: {test_text}")
        print(f"  编码后: {encoded}")
        print(f"  解码后: {decoded}")
        
        # 9. 可选：保存更新后的tokenizer
        # tokenizer.save_pretrained('./model_updated/')  # 取消注释以保存
        # print("\n✅ Tokenizer已保存到 './model_updated/'")
else:
    print("\n没有需要添加的新词")


✅ '<cls>' 不在vocab中，可以添加

成功添加 1 个新词到tokenizer

新添加的词及其ID:
  '<cls>' -> ID: 6400

⚠️  注意：模型嵌入层需要调整大小以支持新词
当前vocab大小: 6401
模型embedding层大小: 6400
是否相同： True
调整后模型embedding层大小: 6401
✅ 已初始化 1 个新token的embedding

验证新词的使用:
  原文: 这是一个测试: <cls> 应该被正确识别
  编码后: [434, 1589, 3560, 28, 223, 6400, 223, 1406, 1183, 2826, 5159]
  解码后: 这是一个测试: <cls> 应该被正确识别


In [13]:
# 在Cell 14的最后，添加保存代码（在tokenizer和model都已更新之后）

import os
import torch

# ========== 保存更新后的tokenizer和模型 ==========

# 1. 设置保存路径
save_base_dir = "./test"  # tokenizer保存目录
save_model_dir = "./test"  # 模型权重保存目录（与init_model的save_dir保持一致）
os.makedirs(save_base_dir, exist_ok=True)
os.makedirs(save_model_dir, exist_ok=True)

# 2. 保存tokenizer到目录
print("\n" + "=" * 80)
print("保存tokenizer...")
print("=" * 80)
tokenizer.save_pretrained(save_base_dir)
print(f"✅ Tokenizer已保存到: {save_base_dir}/")

# 验证tokenizer文件
tokenizer_files = ['tokenizer_config.json', 'vocab.json', 'merges.txt', 'special_tokens_map.json']
for file in tokenizer_files:
    file_path = os.path.join(save_base_dir, file)
    if os.path.exists(file_path):
        print(f"  ✓ {file}")
    else:
        print(f"  ✗ {file} (可能不存在，取决于tokenizer类型)")

# 3. 保存模型权重到.pth文件
print("\n" + "=" * 80)
print("保存模型权重...")
print("=" * 80)

# 获取模型配置参数（用于构建文件名）
# 这些参数需要与init_model时使用的参数一致
model_weight_name = "full_sft_cls"  # 权重文件名（如 'pretrain', 'full_sft' 等）
hidden_size = model.config.hidden_size if hasattr(model, 'config') else 512  # 从模型配置获取
use_moe = getattr(model.config, 'use_moe', False) if hasattr(model, 'config') else False

# 构建权重文件名（格式与init_model期望的一致）
moe_suffix = '_moe' if use_moe else ''
weight_filename = f'{model_weight_name}_{hidden_size}{moe_suffix}.pth'
weight_path = os.path.join(save_model_dir, weight_filename)

# 获取模型状态字典
if hasattr(model, 'module'):  # 如果模型被包装在DataParallel中
    state_dict = model.module.state_dict()
else:
    state_dict = model.state_dict()

# 转换为半精度以节省空间（与训练脚本保持一致）
state_dict_to_save = {k: v.half() if v.dtype == torch.float32 else v 
                      for k, v in state_dict.items()}

# 保存权重
torch.save(state_dict_to_save, weight_path)
print(f"✅ 模型权重已保存到: {weight_path}")
print(f"   权重名称: {model_weight_name}")
print(f"   隐藏层大小: {hidden_size}")
print(f"   MoE后缀: {moe_suffix}")
print(f"   参数量: {sum(p.numel() for p in state_dict.values()) / 1e6:.2f}M")

# 4. 保存模型配置信息（可选，用于记录）
config_info = {
    'vocab_size': len(tokenizer),
    'hidden_size': hidden_size,
    'use_moe': use_moe,
    'new_tokens': tokens_to_add if 'tokens_to_add' in locals() else [],
    'tokenizer_path': save_base_dir,
    'model_weight_path': weight_path,
    'model_weight_name': model_weight_name
}

import json
config_path = os.path.join(save_base_dir, 'model_config_info.json')
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config_info, f, indent=2, ensure_ascii=False)
print(f"✅ 配置信息已保存到: {config_path}")

# 5. 验证保存结果
print("\n" + "=" * 80)
print("保存验证:")
print("=" * 80)

# 验证tokenizer可以重新加载
try:
    test_tokenizer = AutoTokenizer.from_pretrained(save_base_dir)
    print(f"✅ Tokenizer可以正常加载 (vocab大小: {len(test_tokenizer)})")
    # 验证新添加的token是否还在
    if tokens_to_add and all(token in test_tokenizer.get_vocab() for token in tokens_to_add):
        print(f"✅ 新添加的token仍然存在于vocab中")
    del test_tokenizer
except Exception as e:
    print(f"❌ Tokenizer加载失败: {e}")

# 验证模型权重可以加载
try:
    test_weights = torch.load(weight_path, map_location='cpu')
    print(f"✅ 模型权重文件可以正常加载 (包含 {len(test_weights)} 个参数键)")
    del test_weights
except Exception as e:
    print(f"❌ 模型权重加载失败: {e}")

# 6. 保存模型配置到config.json（使得AutoModelForCausalLM.from_pretrained()可以加载）
print("\n" + "=" * 80)
print("保存模型配置...")
print("=" * 80)

# 注册MiniMindConfig和MiniMindForCausalLM到AutoClass
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
MiniMindConfig.register_for_auto_class()
MiniMindForCausalLM.register_for_auto_class("AutoModelForCausalLM")

# 获取模型配置
if hasattr(model, 'config'):
    model_config = model.config
else:
    # 如果没有config，创建一个新的MiniMindConfig
    model_config = MiniMindConfig(
        vocab_size=len(tokenizer),
        hidden_size=hidden_size,
        num_hidden_layers=getattr(model, 'num_hidden_layers', 12),
        use_moe=use_moe
    )

# 更新vocab_size以匹配实际的tokenizer大小
model_config.vocab_size = len(tokenizer)

# 保存配置到config.json
config_path = os.path.join(save_base_dir, 'config.json')
model_config.save_pretrained(save_base_dir)
print(f"✅ 模型配置已保存到: {config_path}")

# 创建modeling_minimind.py文件，用于AutoModelForCausalLM.from_pretrained()加载
modeling_file_path = os.path.join(save_base_dir, 'modeling_minimind.py')
modeling_code = '''# This file is auto-generated for loading MiniMind model with AutoModelForCausalLM
# The actual model implementation is in the project's model/model_minimind.py
import sys
import os

# Add the project root to path so we can import the actual model
project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import and re-export the model classes
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM

# Re-export for transformers
__all__ = ['MiniMindConfig', 'MiniMindForCausalLM']
'''

with open(modeling_file_path, 'w', encoding='utf-8') as f:
    f.write(modeling_code)
print(f"✅ 模型定义文件已创建: {modeling_file_path}")

# 验证config.json是否存在且格式正确
try:
    import json
    with open(config_path, 'r', encoding='utf-8') as f:
        config_dict = json.load(f)
    print(f"✅ config.json验证通过")
    print(f"   模型类型: {config_dict.get('model_type', 'N/A')}")
    print(f"   vocab大小: {config_dict.get('vocab_size', 'N/A')}")
    print(f"   隐藏层大小: {config_dict.get('hidden_size', 'N/A')}")
    
    # 确保model_type字段存在
    if 'model_type' not in config_dict:
        print(f"⚠️  config.json缺少model_type字段，正在修复...")
        config_dict['model_type'] = 'minimind'
        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config_dict, f, indent=2, ensure_ascii=False)
        print(f"✅ 已修复config.json")
except Exception as e:
    print(f"⚠️  config.json验证失败: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 80)
print("保存完成！")
print("=" * 80)
print(f"\n📌 重新加载时使用以下参数:")
print(f"    args.load_from = '{save_base_dir}'")
print(f"    args.save_dir = '{save_model_dir}'")
print(f"    args.weight = '{model_weight_name}'")
print(f"    args.hidden_size = {hidden_size}")
print(f"    args.use_moe = {1 if use_moe else 0}")
print(f"\n✅ 现在可以使用 AutoModelForCausalLM.from_pretrained('{save_base_dir}', trust_remote_code=True) 加载模型")


保存tokenizer...
✅ Tokenizer已保存到: ./test/
  ✓ tokenizer_config.json
  ✗ vocab.json (可能不存在，取决于tokenizer类型)
  ✗ merges.txt (可能不存在，取决于tokenizer类型)
  ✓ special_tokens_map.json

保存模型权重...
✅ 模型权重已保存到: ./test\full_sft_cls_512.pth
   权重名称: full_sft_cls
   隐藏层大小: 512
   MoE后缀: 
   参数量: 29.11M
✅ 配置信息已保存到: ./test\model_config_info.json

保存验证:
✅ Tokenizer可以正常加载 (vocab大小: 6401)
✅ 新添加的token仍然存在于vocab中
✅ 模型权重文件可以正常加载 (包含 75 个参数键)

保存模型配置...
✅ 模型配置已保存到: ./test\config.json
✅ 模型定义文件已创建: ./test\modeling_minimind.py
✅ config.json验证通过
   模型类型: minimind
   vocab大小: 6401
   隐藏层大小: 512

保存完成！

📌 重新加载时使用以下参数:
    args.load_from = './test'
    args.save_dir = './test'
    args.weight = 'full_sft_cls'
    args.hidden_size = 512
    args.use_moe = 0

✅ 现在可以使用 AutoModelForCausalLM.from_pretrained('./test', trust_remote_code=True) 加载模型


In [14]:
# ========== 重新加载保存的tokenizer和模型 ==========

from transformers import AutoTokenizer
from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
import argparse

# 设置加载参数（与保存时保持一致）
class Args:
    def __init__(self):
        self.load_from = "./test"  # tokenizer目录
        self.save_dir = "./test"  # 模型权重目录
        self.weight = "full_sft_cls"  # 权重文件名
        self.hidden_size = 512  # 根据实际模型调整
        self.num_hidden_layers = 12  # 根据实际模型调整
        self.use_moe = 0  # 根据实际模型调整
        self.lora_weight = "None"
        self.inference_rope_scaling = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

# 使用notebook中的init_model函数重新加载
args = Args()

# 重新加载tokenizer和模型
model_reloaded, tokenizer_reloaded = init_model(args)

print("\n" + "=" * 80)
print("重新加载完成！")
print("=" * 80)
print(f"Tokenizer vocab大小: {len(tokenizer_reloaded)}")
print(f"模型参数量: {sum(p.numel() for p in model_reloaded.parameters()) / 1e6:.2f}M")

# 验证新添加的token是否还在
if 'tokens_to_add' in locals() and tokens_to_add:
    print(f"\n验证新添加的token:")
    for token in tokens_to_add:
        if token in tokenizer_reloaded.get_vocab():
            token_id = tokenizer_reloaded.convert_tokens_to_ids(token)
            print(f"  ✅ '{token}' -> ID: {token_id}")
        else:
            print(f"  ❌ '{token}' 不在vocab中")

ValueError: Unrecognized configuration class <class 'transformers_modules.test.model_minimind.MiniMindConfig'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of ApertusConfig, ArceeConfig, AriaTextConfig, BambaConfig, BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BitNetConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, BltConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CohereConfig, Cohere2Config, CpmAntConfig, CTRLConfig, Data2VecTextConfig, DbrxConfig, DeepseekV2Config, DeepseekV3Config, DiffLlamaConfig, DogeConfig, Dots1Config, ElectraConfig, Emu3Config, ErnieConfig, Ernie4_5Config, Ernie4_5_MoeConfig, Exaone4Config, FalconConfig, FalconH1Config, FalconMambaConfig, FlexOlmoConfig, FuyuConfig, GemmaConfig, Gemma2Config, Gemma3Config, Gemma3TextConfig, Gemma3nConfig, Gemma3nTextConfig, GitConfig, GlmConfig, Glm4Config, Glm4MoeConfig, GotOcr2Config, GPT2Config, GPT2Config, GPTBigCodeConfig, G

In [16]:
# 计算bbc_news_train.jsonl中数据转换为训练模板后的最长文本长度
import json

def get_max_training_length(jsonl_path, tokenizer):
    """
    计算训练数据转换为训练模板后的最长文本长度
    
    Args:
        jsonl_path: JSONL文件路径
        tokenizer: tokenizer对象
    
    Returns:
        max_length: 最长文本的token长度
        max_index: 最长文本的样本索引
        max_sample: 最长文本的样本内容
    """
    max_length = 0
    max_index = -1
    max_sample = None
    all_lengths = []
    
    print(f"正在处理文件: {jsonl_path}")
    print("=" * 80)
    
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            try:
                data = json.loads(line.strip())
                
                # 检查数据格式
                if 'conversations' not in data:
                    print(f"⚠️  行 {line_num}: 缺少 'conversations' 字段，跳过")
                    continue
                
                conversations = data['conversations']
                if not conversations or len(conversations) == 0:
                    print(f"⚠️  行 {line_num}: conversations 为空，跳过")
                    continue
                
                # 使用与SFTDataset相同的方式创建训练模板
                # 参考 SFTDataset._create_chat_prompt 方法
                messages = conversations.copy()
                tools = conversations[0]["functions"] if (conversations and conversations[0]["role"] == "system" and conversations[0].get("functions")) else None
                
                # 应用chat_template（与训练时保持一致）
                prompt = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=False,
                    tools=tools
                )
                
                # 对转换后的文本进行tokenize
                input_ids = tokenizer(prompt, add_special_tokens=False).input_ids
                length = len(input_ids)
                all_lengths.append(length)
                
                # 更新最大值
                if length > max_length:
                    max_length = length
                    max_index = line_num - 1  # 转换为0-based索引
                    max_sample = {
                        'index': max_index,
                        'line_num': line_num,
                        'length': length,
                        'conversations': conversations,
                        'prompt_preview': prompt[:200] + '...' if len(prompt) > 200 else prompt
                    }
                
                # 每处理100条数据打印一次进度
                if line_num % 100 == 0:
                    print(f"已处理 {line_num} 条数据，当前最长长度: {max_length}")
                    
            except json.JSONDecodeError as e:
                print(f"⚠️  行 {line_num}: JSON解析错误 - {e}")
                continue
            except Exception as e:
                print(f"⚠️  行 {line_num}: 处理错误 - {e}")
                continue
    
    return max_length, max_index, max_sample, all_lengths

# 执行计算（需要先运行Cell 10初始化tokenizer）
if 'tokenizer' in globals():
    max_length, max_index, max_sample, all_lengths = get_max_training_length('dataset/bbc_news_data.jsonl', tokenizer)
    
    print("\n" + "=" * 80)
    print("结果统计:")
    print("=" * 80)
    print(f"总样本数: {len(all_lengths)}")
    print(f"最长训练文本长度: {max_length} tokens")
    print(f"最长文本的样本索引: {max_index} (行号: {max_sample['line_num']})")
    print(f"\n长度统计:")
    print(f"  平均长度: {sum(all_lengths) / len(all_lengths):.2f} tokens")
    print(f"  最短长度: {min(all_lengths)} tokens")
    print(f"  最长长度: {max(all_lengths)} tokens")
    print(f"  中位数长度: {sorted(all_lengths)[len(all_lengths)//2]} tokens")
    
    # 显示最长样本的详细信息
    if max_sample:
        print(f"\n最长样本详情:")
        print(f"  索引: {max_sample['index']}")
        print(f"  行号: {max_sample['line_num']}")
        print(f"  长度: {max_sample['length']} tokens")
        print(f"  User内容预览: {max_sample['conversations'][0]['content'][:100]}...")
        print(f"  Assistant内容: {max_sample['conversations'][1]['content']}")
        print(f"  训练模板预览: {max_sample['prompt_preview']}")
    
    # 长度分布统计
    print(f"\n长度分布:")
    # 从0-2000每隔100设置一个区间
    length_ranges = [(0,1024,'0-1024')]
    for start in range(0, 2000, 100):
        end = start + 100
        label = f"{start}-{end}"
        length_ranges.append((start, end, label))
    # 添加最后一个区间：2000+
    length_ranges.append((2000, float('inf'), "2000+"))
    
    for min_len, max_len, label in length_ranges:
        count = sum(1 for l in all_lengths if min_len <= l < max_len)
        percentage = count / len(all_lengths) * 100 if all_lengths else 0
        print(f"  {label:10s}: {count:4d} 条 ({percentage:5.2f}%)")
else:
    print("⚠️  请先运行 Cell 10 初始化 tokenizer")


正在处理文件: dataset/bbc_news_data.jsonl
已处理 100 条数据，当前最长长度: 1266
已处理 200 条数据，当前最长长度: 1525
已处理 300 条数据，当前最长长度: 1702
已处理 400 条数据，当前最长长度: 1708
已处理 500 条数据，当前最长长度: 1708
已处理 600 条数据，当前最长长度: 1708
已处理 700 条数据，当前最长长度: 2217
已处理 800 条数据，当前最长长度: 6270
已处理 900 条数据，当前最长长度: 6270
已处理 1000 条数据，当前最长长度: 6270
已处理 1100 条数据，当前最长长度: 6270
已处理 1200 条数据，当前最长长度: 7710
已处理 1300 条数据，当前最长长度: 7710
已处理 1400 条数据，当前最长长度: 7710
已处理 1500 条数据，当前最长长度: 7710
已处理 1600 条数据，当前最长长度: 7710
已处理 1700 条数据，当前最长长度: 7710
已处理 1800 条数据，当前最长长度: 7710
已处理 1900 条数据，当前最长长度: 7710
已处理 2000 条数据，当前最长长度: 7710
已处理 2100 条数据，当前最长长度: 7710
已处理 2200 条数据，当前最长长度: 7710

结果统计:
总样本数: 2225
最长训练文本长度: 7710 tokens
最长文本的样本索引: 1185 (行号: 1186)

长度统计:
  平均长度: 800.13 tokens
  最短长度: 259 tokens
  最长长度: 7710 tokens
  中位数长度: 708 tokens

最长样本详情:
  索引: 1185
  行号: 1186
  长度: 7710 tokens
  User内容预览: 标题：Terror powers expose 'tyranny'
内容： The Lord Chancellor has defended government plans to introduce...
  Assistant内容: politics
  训练模板预览: <|im_start|>system
You are a helpful assistant<